In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import emd
import tarfile
import io
import scipy
from scipy import signal
from icecream import ic
import neurodsp.filt as dsp
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

# Directories
Set the working directory as well as access to the main LFP dataset and supplementary dataset

In [ ]:
# Set the current working directory
cwd = os.chdir(r"C:\Python Work Directory\NMA_Impact_Scholars_Steinmetz")

# Access to the Steinmetz LFP dataset
# lfp_dat = r"E:\Steinmetz_Dataset"
lfp_dat  = r"C:\Python Work Directory\NMA_Impact_Scholars_Steinmetz\data\examples"

# @title Data retrieval
data_directory = r'data\spikeAndBehavioralData'

# test_dataset
test_LFP = r"Cori_2016-12-18"

In [ ]:
print(os.path.join(os.getcwd(),data_directory))

## Brain Regions of Interest

In [ ]:
hpc = ["CA1", "CA3", "DG", "SUB"]
pfc = ["ACA", "ILA", "PL","RSP"]
region_loop = hpc + pfc
region_select = 'CA1'

## Power spectrum functions

### Defining file iterator (for later use)

In [ ]:
walker = os.walk(os.path.join(os.getcwd(),data_directory))
for root, dirs, files in walker:
    print(root)
    print(dirs)
    print(files)

### .npy file loader from tarball

In [ ]:
def npy_loader(filename:str)-> np.ndarray:
    '''
    Numpy loader function for .npy in tarball (.tar) packages.
    
    :param filename: str
    :return: np.ndarray 
    '''
    try:
        npy_file = tar.extractfile(filename)
        if npy_file is not None:
            npy_file_content = npy_file.read()
            
            # Check file size to confirm it's not empty or corrupted
            if len(npy_file_content) == 0:
                raise ValueError(f"The .npy file '{filename}' is empty or corrupted.")
            
            # Load .npy file from memory using BytesIO
            np_data = np.load(io.BytesIO(npy_file_content))
            return np_data
        else:
            raise FileNotFoundError(f"Could not find or extract the file: {probe_filename}")
    except Exception as e:
        print(f"Error reading .npy file: {e}")
    

In [ ]:
alldata_tar_path = os.path.join(os.getcwd(),data_directory,test_LFP + r".tar")
with tarfile.open(alldata_tar_path, 'r') as tar:
    print(tar.getnames())
    
    brain_loc_filename = [name for name in tar.getnames()[:5] if name.endswith('.tsv')][0]
    probe_desc_filename = [name for name in tar.getnames() if name.endswith('rawFilename.tsv')][0]
    probe_filename = [name for name in tar.getnames() if name.endswith('channels.probe.npy')][0]
    raw_Row_filename = [name for name in tar.getnames() if name.endswith('channels.rawRow.npy')][0]
    site_filename = [name for name in tar.getnames() if name.endswith('channels.site.npy')][0]
    site_pos_filename = [name for name in tar.getnames() if name.endswith('channels.sitePositions.npy')][0]
    
    
    brain_loc = pd.read_csv(tar.extractfile(brain_loc_filename), sep='\t')
    probe_desc = pd.read_csv(tar.extractfile(probe_desc_filename), sep='\t')
    probe = npy_loader(probe_filename)
    raw_Row = npy_loader(raw_Row_filename)
    site = npy_loader(site_filename)
    site_pos = npy_loader(site_pos_filename)
    
    

In [ ]:
brain_loc.shape

In [ ]:
brain_loc.query(f'allen_ontology == "{region_select}"')

In [ ]:
probe_desc

In [ ]:
brain_loc['probe'] = probe
brain_loc['site'] = site
brain_loc[['site_pos_x','site_pos_y']] = site_pos
brain_loc['raw_Row'] = raw_Row

In [ ]:
brain_loc.query(f'allen_ontology == "{region_select}"') 

## Discovering the Channel Labelling Scheme

In [ ]:
# Create the scatter plot using Plotly Express
fig = px.scatter(brain_loc.query('probe == 0'),
                 x='site_pos_x',
                 y='site_pos_y',
                 color='site',
                 title='Brain Location Scatter Plot',
                 width=1200,  # Equivalent to figsize=(20,10)
                 height=600)

# Customize the layout if needed
fig.update_layout(
    title_x=0.5,  # Center the title
    legend_title_text='Site',
    # Add any additional layout customizations here
)

# Show the plot
fig.show()

In [ ]:
# Create the scatter plot using Plotly Express
fig = px.scatter(brain_loc.query('probe == 0'),
                 x='site_pos_x',
                 y='site_pos_y',
                 color='allen_ontology',
                 title='Brain Location Scatter Plot',
                 width=1200,  # Equivalent to figsize=(20,10)
                 height=600)

# Customize the layout if needed
fig.update_layout(
    title_x=0.5,  # Center the title
    legend_title_text='Site',
    # Add any additional layout customizations here
)

# Show the plot
fig.show()

### Probe Selection

Select the necessary probes that have recording sites of our brain regions of interest

In [ ]:
# Identify probe for CA1
probe_select = brain_loc.query(f'allen_ontology == "{region_select}"')['probe'].unique() == np.array(probe_desc.index)

In [ ]:
#TODO: Build a dataset loader that interacts with the online database

# Path to your .tar file


tar_path = os.path.join(lfp_dat,test_LFP + r"_lfp.tar")



# Define the parameters based on the documentation
num_channels = 385  # 385 channels as specified
data_type = np.int16  # int16 data type
sampling_rate = 2500  # 2500 Hz sampling rate

# Open the .tar file and load the .bin file
with tarfile.open(tar_path, 'r') as tar:
    # Identify the .bin file (assuming there's only one)
    bin_file_name = np.array(tar.getnames())[probe_select][0]
    
    
    # Extract the .bin file to memory
    bin_file = tar.extractfile(bin_file_name)
    
    # Determine the number of samples by dividing the file size by the number of channels
    # and the size of each data point (2 bytes for int16)
    file_size = tar.getmember(bin_file_name).size
    num_samples = file_size // (num_channels * np.dtype(data_type).itemsize)
    
    # Read the .bin file in chunks if it's too large for memory
    chunk_size = 1000000  # Set a reasonable chunk size
    all_data = []
    
    while True:
        # Read a chunk of data
        data_chunk = np.frombuffer(bin_file.read(chunk_size * num_channels * np.dtype(data_type).itemsize), dtype=data_type)
        if data_chunk.size == 0:
            break
        # Reshape the chunk to (num_channels, chunk_samples)
        data_chunk = data_chunk.reshape(-1, num_channels).T
        all_data.append(data_chunk)
    
    # Concatenate all chunks if the entire data needs to be loaded
    reshaped_data = np.hstack(all_data)

# At this point, reshaped_data contains the LFP data in shape (385, num_total_samples)

In [ ]:
ic(reshaped_data.shape)

## Synchronization Signal Channel
When plotting Channel 385, we can observe that this channel contains our time events of stimulus being presented

In [ ]:
sampling_rate = 2500
total_time = reshaped_data.shape[1]/sampling_rate
time_points = np.linspace(0,total_time, reshaped_data.shape[1])
time_points_ms = time_points*1000

sync_signal_fig = px.line(
    x=time_points_ms[:1000000],
    y=reshaped_data[-1,:1000000],
    labels={'x': 'Time (ms)', 'y': 'Amplitude (μV)'},
    title='Synchronization Signal Time Series'
)

sync_signal_fig.show()

### Plot of a random CA1 channel

In [ ]:
CA1_signal_fig = px.line(
    x=time_points_ms[:1000000],
    y=reshaped_data[-233,:1000000],
    labels={'x': 'Time (ms)', 'y': 'Amplitude (μV)'},
    title='CA1, Channel 233 Signal Time Series'
)

CA1_signal_fig.show()

### Power Spectrum of CA1


In [ ]:
select_channels = reshaped_data[brain_loc.query(f'allen_ontology == "{region_select}"')['raw_Row'].unique()]

In [ ]:
freqs, pspec = signal.welch(x = select_channels, fs = 2500, scaling = 'spectrum', nperseg = 4*1024)

In [ ]:
print(pspec[1].shape)

# Create the figure
fig = go.Figure(
    data=go.Scatter(
        x=freqs,  # x-axis as index if no specific frequency array
        y=pspec,  # power spectrum values
        mode='lines',
        line=dict(color='blue')
    )
)

# Customize layout
fig.update_layout(
    title='Power Spectrum',
    width=1200,  # Approximates 20 inches
    height=600,  # Approximates 10 inches
    xaxis_title='Frequency (Hz)',
    yaxis_title='Power',
    yaxis_type='log'  # Equivalent to semilogy
)

# Display the plot
fig.show()

In [ ]:
plot_power_spectrum(reshaped_data[233,:],2500,scaling='spectrum')

In [ ]:
plot_power_spectrum(reshaped_data[0,:],2500)

In [ ]:
# Retrieves the reference to subtract from the signal
def CAR_filter(signal, mode ='mean'):
    avg_ref = np.zeros((signal.shape[0],1))
    if mode == 'mean':
        avg_ref = np.mean(signal,axis=0)
    if mode == 'median':
        avg_ref = np.median(signal,axis=0)
    return avg_ref

In [ ]:
avg_ref = CAR_filter(reshaped_data[:-1], mode='median')

### Selecting regions of interests
We would like to look at CA1 region in the hippocampus as an exercise first

In [ ]:
select_channels = reshaped_data[brain_loc.query(f'allen_ontology == "{region_select}"')['raw_Row'].unique()] - avg_ref


In [ ]:
freqs, power_spectrum = plot_brain_power_spectrum(select_channels,2500, nperseg=4*1024, scaling='spectrum', norm=False)

In [ ]:
print(freqs.shape)
print(power_spectrum.shape)

In [ ]:
print(select_channels.shape)

In [ ]:
plt.gcf().set_size_inches(20, 10)
for psd in power_spectrum:
    plt.semilogy(freqs,psd)
plt.ylabel('Power')
plt.xlabel('Frequency (Hz)')

In [ ]:
print(np.mean(select_channels,axis=0).shape)

### CA1 Channels Mean and Standard Deviations

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(np.mean(select_channels,axis=0)[:2500], linestyle='dashed', color='blue')
plt.fill_between(range(len(np.mean(select_channels,axis=0)[:2500])), np.mean(select_channels,axis=0)[:2500]+2*np.std(select_channels,axis=0)[:2500],np.mean(select_channels,axis=0)[:2500]-2*np.std(select_channels,axis=0)[:2500],alpha=0.3)

In [ ]:
# A highpass filter to isolate higher frequencies in each channel

select_filtered_channels = dsp.filter_signal(select_channels,2500,'highpass',(150,250), filter_type='iir',butterworth_order=3)


In [ ]:
print(select_filtered_channels.shape)

### Comparison of a raw CA1 LFP with respect to higher frequencies

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(select_channels[0,:2500])
plt.plot(select_filtered_channels[0,:2500])
plt.xlabel('Samples')
plt.ylabel('Amplitude (uV)')

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(np.mean(select_filtered_channels,axis=0)[:2500],linestyle='dashed', color='blue')
plt.fill_between(range(len(np.mean(select_filtered_channels,axis=0)[:2500])), np.mean(select_filtered_channels,axis=0)[:2500]+2*np.std(select_filtered_channels,axis=0)[:2500],np.mean(select_filtered_channels,axis=0)[:2500]-2*np.std(select_filtered_channels,axis=0)[:2500],alpha=0.3)
plt.xlabel('Samples')
plt.ylabel('Amplitude (uV)')

In [ ]:
# Root mean square function for calculating max ripple power
def rms (signal):
    rms = np.sqrt(np.mean(np.square(signal),axis=1))
    return rms

In [ ]:
ripple_power = rms(select_filtered_channels)

In [ ]:
optimum_channel = np.where(ripple_power==np.max(ripple_power))[0][0]
print(optimum_channel)

### Power spectrum after selecting the best HPC channel

In [ ]:
plot_power_spectrum(select_channels[optimum_channel,:],2500)

In [ ]:
# Lowpass filter to cutoff frequencies above 200 Hz

filtered_LFP = dsp.filter_signal(select_channels[optimum_channel],2500,'lowpass',(None,200), filter_type='iir',butterworth_order=4)

In [ ]:
fig_CA1,ax_CA1 = plt.subplots(4,1,figsize=(20,15), sharex=True)
ax_CA1[0].plot(select_channels[0, :2500])
ax_CA1[0].title.set_text('CA1 top site')
ax_CA1[1].plot(select_channels[optimum_channel, :2500])
ax_CA1[1].title.set_text('CA1 probe site 251')
ax_CA1[2].plot(filtered_LFP[:2500])
ax_CA1[2].title.set_text('Low-Pass filtered site 251')
ax_CA1[3].plot(select_channels[0, :2500])
ax_CA1[3].title.set_text('CA1 bottom site')
plt.xlabel('Samples')
plt.ylabel('Amplitude (uV)')

In [ ]:
np.savez(os.path.join(lfp_dat,test_LFP + r"_CH251"), data = select_channels[optimum_channel,:])

In [ ]:
new_sig = np.load(os.path.join(lfp_dat,test_LFP + r"_CH251.npz"))['data']
print(new_sig.shape)
                  

In [ ]:
plot_power_spectrum(filtered_LFP,2500)

In [ ]:
# Import Theta Cycle Extraction and Data Management Class Functions
from src.signal import *
from src.functions import peak_cog, fpp_peaks


In [ ]:
# Intialize with selecting the optimum channel RAW LFP
wake_test_one = WakeSignal(select_channels[optimum_channel],2500,(5,12))

In [ ]:
print(wake_test_one.mask_freq)

In [ ]:
emd.plotting.plot_imfs(wake_test_one.imf[2500:5000])

In [ ]:
sub, theta, gamma = tg_split(wake_test_one.mask_freq,wake_test_one.freq_range)
print(theta)

In [ ]:
wake_test_one.frequency_transform('hilbert')

In [ ]:
fig_freq,ax_freq=plt.subplots(1,2,figsize=(15,5))
colors = ['blue','green','red', 'yellow']
## Iterated Mask Sift 
for i, thet in enumerate(wake_test_one.mask_freq[theta]):
    ax_freq[0].hist(wake_test_one.IA.T[theta][i],bins=100,edgecolor='none',alpha=0.3 + (0.2*i),color= colors[i],label = str(np.round(thet,2))+ 'Hz',density=True)
ax_freq[0].legend(loc='upper right')

for j, gam in enumerate(wake_test_one.mask_freq[gamma]):
    ax_freq[1].hist(wake_test_one.IA.T[gamma][j],bins=100,edgecolor='none',alpha=0.3 + (0.2*j),color=colors[j],label = str(np.round(gam,2))+ 'Hz',density=True)
ax_freq[1].legend(loc='upper right')

ax_freq[0].set_title(' Theta Instantaneous Amplitude Distribution')
ax_freq[1].set_title('Gamma Instantaneous Amplitude Distribution')
ax_freq[0].set_xlabel(r'Amplitude (uV)')
ax_freq[1].set_xlabel(r'Amplitude (uV)')
fig_freq.suptitle('Raw Amplitude Distribution Across imfs')
fig_freq.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_RawAmplitude_Distribution.png'))

In [ ]:
# Extract cycles that satisfy thresholds
fpp_cycles_one = wake_test_one.get_fpp_cycles(frequencies=(15,120), band = 'gamma', norm='zscore', mode = 'power')

In [ ]:
print(fpp_cycles_one.shape)

In [ ]:
# Intialize with selecting the optimum channel RAW LFP minus the high frequency components using EMD
wake_test_two = WakeSignal(np.sum(wake_test_one.imf[:,1:],axis=1),2500,(5,12))

In [ ]:
print(wake_test_two.mask_freq)

In [ ]:
emd.plotting.plot_imfs(wake_test_two.imf[2500:5000])

In [ ]:
wake_test_two.frequency_transform('hilbert')

In [ ]:
sub, theta, gamma = tg_split(wake_test_two.mask_freq,wake_test_two.freq_range)

In [ ]:
fig_freq2, ax_freq2 = plt.subplots(1, 2, figsize=(15, 5))
colors = ['blue', 'green', 'red']
## Iterated Mask Sift 
for i, thet in enumerate(wake_test_two.mask_freq[theta]):
    ax_freq2[0].hist(wake_test_two.IA.T[theta][i], bins=100, edgecolor='none', alpha=0.3 + (0.2 * i), color=colors[i],
                    label=str(np.round(thet, 2)) + 'Hz', density=True)
ax_freq2[0].legend(loc='upper right')

for j, gam in enumerate(wake_test_two.mask_freq[gamma]):
    ax_freq2[1].hist(wake_test_two.IA.T[gamma][j], bins=100, edgecolor='none', alpha=0.3 + (0.2 * j), color=colors[j],
                    label=str(np.round(gam, 2)) + 'Hz', density=True)
ax_freq2[1].legend(loc='upper right')

ax_freq2[0].set_title(' Theta Instantaneous Amplitude Distribution')
ax_freq2[1].set_title('Gamma Instantaneous Amplitude Distribution')
ax_freq2[0].set_xlabel(r'Amplitude (uV)')
ax_freq2[1].set_xlabel(r'Amplitude (uV)')
fig_freq2.suptitle('High Freq Removal Amplitude Distribution Across imfs')
fig_freq2.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_HighFreqRemovalAmplitude_Distribution.png'))

In [ ]:
fpp_cycles_two = wake_test_two.get_fpp_cycles(frequencies=(15,120), band = 'gamma',norm='zscore', mode = 'power')

In [ ]:
# Intialize with selecting the using low pass filtered LFP
wake_test_three = WakeSignal(filtered_LFP,2500,(5,12))
fpp_cycles_three = wake_test_three.get_fpp_cycles(frequencies=(15,120), band = 'gamma', norm='zscore', mode = 'power')

In [ ]:
print(wake_test_three.mask_freq)

In [ ]:
wake_test_three.frequency_transform('hilbert')

In [ ]:
sub, theta, gamma = tg_split(wake_test_three.mask_freq,wake_test_three.freq_range)

In [ ]:
fig_freq3, ax_freq3 = plt.subplots(1, 2, figsize=(15, 5))
colors = ['blue', 'green', 'red']
## Iterated Mask Sift 
for i, thet in enumerate(wake_test_three.mask_freq[theta]):
    ax_freq3[0].hist(wake_test_three.IA.T[theta][i], bins=100, edgecolor='none', alpha=0.3 + (0.2 * i), color=colors[i],
                    label=str(np.round(thet, 2)) + 'Hz', density=True)
ax_freq3[0].legend(loc='upper right')

for j, gam in enumerate(wake_test_three.mask_freq[gamma]):
    ax_freq3[1].hist(wake_test_three.IA.T[gamma][j], bins=100, edgecolor='none', alpha=0.3 + (0.2 * j), color=colors[j],
                    label=str(np.round(gam, 2)) + 'Hz', density=True)
ax_freq3[1].legend(loc='upper right')

ax_freq3[0].set_title(' Theta Instantaneous Amplitude Distribution')
ax_freq3[1].set_title('Gamma Instantaneous Amplitude Distribution')
ax_freq3[0].set_xlabel(r'Amplitude (uV)')
ax_freq3[1].set_xlabel(r'Amplitude (uV)')
fig_freq3.suptitle('Raw Amplitude Distribution Across imfs')
fig_freq3.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_LowPassFiltAmplitude_Distribution.png'))

In [ ]:
# Generate center of gravity peaks for each cycle
peaks_one = wake_test_one.peak_center_of_gravity(frequencies=(15,120), band = 'gamma', norm='zscore', mode = 'power')
peaks_two = wake_test_two.peak_center_of_gravity(frequencies=(15,120), band = 'gamma', norm='zscore', mode = 'power')
peaks_three = wake_test_three.peak_center_of_gravity(frequencies=(15,120), band = 'gamma', norm='zscore', mode = 'power')

In [ ]:
# Generate all peaks from FPP topology
fpp_peaks_one = wake_test_one.get_fpp_peaks(frequencies=(15,120))
fpp_peaks_two = wake_test_two.get_fpp_peaks(frequencies=(15,120))
fpp_peaks_three = wake_test_three.get_fpp_peaks(frequencies=(15,120))

In [ ]:
print(fpp_cycles_two.shape)
print(fpp_cycles_three.shape)

In [ ]:
frequencies = np.arange(15,121,1)
angles = np.linspace(-180,180,19)
print(angles)

In [ ]:
index = 12384
fig,ax = plt.subplots(1,3,figsize=(15,8), sharey=True)
ax[0].set_ylabel('Frequency (Hz)')
ax[0].contourf(angles,frequencies,fpp_cycles_one[index,:,:], vmin=-2, vmax=6,cmap ='viridis')
ax[0].scatter(fpp_peaks_one[index][:,1],fpp_peaks_one[index][:,0], color='blue')
ax[0].scatter(peaks_one[index,1],peaks_one[index,0], color='red',marker='+',s=50)
ax[1].contourf(angles,frequencies,fpp_cycles_two[index,:,:], vmin=-2, vmax=6,cmap ='viridis')
ax[1].scatter(fpp_peaks_two[index][:,1],fpp_peaks_two[index][:,0], color='blue')
ax[1].scatter(peaks_two[index,1],peaks_two[index,0], color='red', marker='+',s=50)
ax[2].contourf(angles,frequencies,fpp_cycles_three[index,:,:], vmin=-2, vmax=6,cmap ='viridis')
ax[2].scatter(fpp_peaks_three[index][:,1],fpp_peaks_three[index][:,0], color='blue')
ax[2].scatter(peaks_three[index,1],peaks_three[index,0], color='red', marker='+',s=50)
ax[0].set_xlabel('Phase (degrees)')
ax[1].set_xlabel('Phase (degrees)')
ax[2].set_xlabel('Phase (degrees)')
plt.tight_layout()

In [ ]:
spectral_signatures_one = np.mean(fpp_cycles_one,axis=2)
spectral_signatures_two = np.mean(fpp_cycles_two,axis=2)
spectral_signatures_three = np.mean(fpp_cycles_three,axis=2)

In [ ]:
peaks_one_df = pd.DataFrame(columns=['frequency','phase'])
peaks_one_df['frequency'] = peaks_one[:,0]
peaks_one_df['phase'] = peaks_one[:,1]
g = sns.JointGrid(data=peaks_one_df, x="phase", y="frequency")
g = g.plot_joint(sns.kdeplot)
g = g.plot_marginals(sns.kdeplot, fill=True)
g.fig.suptitle("CoG Gamma Frequency Density Plot of Theta Cycles Raw LFPs")
g.fig.subplots_adjust(top=0.9)
g.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_CoG_Raw.png'))

In [ ]:
peaks_two_df = pd.DataFrame(columns=['frequency','phase'])
peaks_two_df['frequency'] = peaks_two[:,0]
peaks_two_df['phase'] = peaks_two[:,1]
g = sns.JointGrid(data=peaks_two_df, x="phase", y="frequency")
g = g.plot_joint(sns.kdeplot)
g = g.plot_marginals(sns.kdeplot, fill=True)
g.fig.suptitle("CoG Gamma Frequency Density Plot of Theta Cycles IMF High Freq removal")
g.fig.subplots_adjust(top=0.9)
g.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_CoG_High_Freq_Removal.png'))

In [ ]:
peaks_three_df = pd.DataFrame(columns=['frequency','phase'])
peaks_three_df['frequency'] = peaks_three[:,0]
peaks_three_df['phase'] = peaks_three[:,1]
g = sns.JointGrid(data=peaks_three_df, x="phase", y="frequency")
g = g.plot_joint(sns.kdeplot)
g = g.plot_marginals(sns.kdeplot, fill=True)
g.fig.suptitle("CoG Gamma Frequency Density Plot of Theta Cycles Low Pass Filter")
g.fig.subplots_adjust(top=0.9)
g.savefig(os.path.join(os.getcwd(),r'figures',f'{region_select}_CoG_Low_Pass_Filter.png'))

In [ ]:
def find_peaks_bool_array(signals):
    """
    Find all relative maxima in an array of 1D signals and return a boolean array for each signal.

    Parameters:
    - signals: A 2D numpy array where each row represents a signal.

    Returns:
    - A 2D boolean array with the same shape as 'signals', where True indicates a peak and False otherwise
      for each signal in the input array.
    """
    num_signals, signal_length = signals.shape

    # Initialize a 2D boolean array with False values
    peaks_bool_array = np.zeros_like(signals, dtype=bool)

    for i in range(num_signals):
        # Find relative maxima indices using argrelmax for each signal
        maxima_indices = signal.argrelmax(signals[i])[0]

        # Set the indices of relative maxima to True for the current signal
        peaks_bool_array[i, maxima_indices] = True

    return peaks_bool_array

# Example usage with an array of signals:

In [ ]:
freq_signatures_one = find_peaks_bool_array(spectral_signatures_one)*frequencies
freq_signatures_two = find_peaks_bool_array(spectral_signatures_two)*frequencies
freq_signatures_three = find_peaks_bool_array(spectral_signatures_three)*frequencies

freq_signatures_one = freq_signatures_one.flatten()
freq_signatures_two = freq_signatures_two.flatten()
freq_signatures_three =freq_signatures_three.flatten()

freq_signatures_one = freq_signatures_one[freq_signatures_one > 0]
freq_signatures_two = freq_signatures_two[freq_signatures_two > 0]
freq_signatures_three = freq_signatures_three[freq_signatures_three > 0]

In [ ]:
fig1, axes = plt.subplots(1, 3, figsize=(16, 8))  # Adjust the figure size as needed
sns.set(style="whitegrid")

sns.kdeplot(y=freq_signatures_one, ax=axes[0], fill=True)
axes[0].set_ylabel("Density")
axes[0].set_title("Kernel Density Plot of Frequencies")

sns.kdeplot(y=freq_signatures_two, ax=axes[1], fill=True)
axes[1].set_ylabel("Density")
axes[1].set_title("Kernel Density Plot of Frequencies")

sns.kdeplot(y=freq_signatures_three, ax=axes[2], fill=True)
axes[2].set_ylabel("Density")
axes[2].set_title("Kernel Density Plot of Frequencies")

fig1.suptitle('Kernel Density Plot of Frequencies')

# Adjust the spacing between subplots
plt.tight_layout()

plt.show()